## GOLD TO API LAYER

### API Layer


In [ ]:
# Importing Common Utility Function
import json

import pandas as pd

from StockETL import DateTimeUtil, GlobalPath

In [ ]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [ ]:
# Define file paths using GlobalPath
current_holding_records_file_path = GlobalPath(
    f"DATA/GOLD/Holding/CurrentHolding_data.csv"
)
stockprice_silver_file_path = GlobalPath("DATA/SILVER/StockPrice/StockPrice_data.csv")
holding_gold_file_path = GlobalPath(f"DATA/GOLD/Holding/Holding_data.csv")
profitloss_gold_file_path = GlobalPath(f"DATA/GOLD/ProfitLoss/ProfitLoss_data.csv")
dividend_gold_file_path = GlobalPath(f"DATA/GOLD/Dividend/Dividend_data.csv")

In [ ]:
api_outputs = {}

In [ ]:
# Main flow
df_current_holding = pd.read_csv(current_holding_records_file_path)
print(f"Processing Data => {current_holding_records_file_path}")

# Processes stock prices to extract the latest closing price for each symbol
df_stockprice = pd.read_csv(stockprice_silver_file_path)
print(f"Processing Data => {stockprice_silver_file_path}")

df_stockprice["date"] = pd.to_datetime(df_stockprice["date"])
df_stockprice["close_price"] = df_stockprice["close"]
idx = df_stockprice.groupby("symbol")["date"].idxmax()
df_stockprice = df_stockprice.loc[idx].reset_index(drop=True)

# Merges holding data with stock prices and calculates PnL
df_current_holding = pd.merge(
    df_current_holding,
    df_stockprice[["symbol", "close_price"]],
    on="symbol",
    how="left",
)
df_current_holding["close_amount"] = (
    df_current_holding["close_price"] * df_current_holding["quantity"]
)
df_current_holding["pnl_amount"] = (
    df_current_holding["close_amount"] - df_current_holding["amount"]
)
df_current_holding = df_current_holding.round(2)

api_outputs["current_holding_data"] = (
    df_current_holding.groupby("username")[
        [col for col in df_current_holding.columns if col != "username"]
    ]
    .apply(lambda x: x.to_dict("records"))
    .to_dict()
)

In [ ]:
df_holding = pd.read_csv(holding_gold_file_path)
print(f"Processing Data => {holding_gold_file_path}")

# Processes holding history data to calculate trends
df_holding["date"] = pd.to_datetime(df_holding["date"]).dt.date
df_holding = (
    df_holding.groupby(["username", "date"])[
        [
            "holding_amount",
            "open_amount",
            "high_amount",
            "low_amount",
            "close_amount",
        ]
    ]
    .sum()
    .reset_index()
)

df_holding = df_holding.round(2).rename(
    columns={col: col.replace("_amount", "") for col in df_holding.columns}
)

df_holding = (
    df_holding[["username", "date", "open", "high", "low", "close", "holding"]]
    .sort_values(by=["date"])
    .reset_index(drop=True)
)

api_outputs["holding_trands_data"] = (
    df_holding.groupby("username")[
        [col for col in df_holding.columns if col != "username"]
    ]
    .apply(lambda x: x.to_dict("records"))
    .to_dict()
)

In [ ]:
df_pnl = pd.read_csv(profitloss_gold_file_path)
print(f"Processing Data => {profitloss_gold_file_path}")


api_outputs["profit_loss_data"] = (
    df_pnl.groupby("username")[[col for col in df_pnl.columns if col != "username"]]
    .apply(lambda x: x.to_dict("records"))
    .to_dict()
)

In [ ]:
df_dividend = pd.read_csv(dividend_gold_file_path)
print(f"Processing Data => {dividend_gold_file_path}")
api_outputs["dividend_data"] = (
    df_dividend.groupby("username")[
        [col for col in df_dividend.columns if col != "username"]
    ]
    .apply(lambda x: x.to_dict("records"))
    .to_dict()
)

In [ ]:
# Create the final output dictionary for the API
load_timestamp = DateTimeUtil.today()


for api_name, output in api_outputs.items():
    api_file_path = GlobalPath(f"DATA/API/{api_name}.json")
    output = {"data": output, "load_timestamp": load_timestamp}

    # Clean the output
    output = replace_nan_with_empty(output)

    # Write the result to a JSON file
    with open(api_file_path, "w", encoding="utf-8") as json_file:
        json.dump(
            output,
            json_file,
            indent=4,
            allow_nan=False,
            ensure_ascii=True,
            default=str,
            # sort_keys=True,
        )
        json_file.write("\n")  # Add newline at the end of the file for clean formatting
    print(f"Data written to => {api_file_path}")